## Data Exploration and Preprocessing Pipeline

In [75]:
# Imports
import musdb
import numpy as np 
import matplotlib.pyplot as plt 

In [47]:
# Loading the dataset
db = musdb.DB(root="Dataset", download=True)

In [77]:
mus_train = musdb.DB(root='./Dataset', subsets="train")
mus_test = musdb.DB(root='./Dataset', subsets="test")

In [49]:
for track in mus_train[:3]:
    # print(f"Name: {track.name} Instruments: {track.stems} Targets: {track.targets}")
    print(track.stems.shape)

(5, 300032, 2)
(5, 300032, 2)
(5, 300032, 2)
(5, 300032, 2)
(5, 300032, 2)


In [50]:
from IPython.display import Audio, display
import numpy as np
import ipywidgets as widgets

track = mus_train[0]  
instruments = list(track.targets.keys())

first_audio = track.targets[instruments[0]].audio
sr = track.rate
audio_len = first_audio.shape[0] if hasattr(first_audio, 'shape') else len(first_audio)
duration_sec = int(audio_len // sr)

dropdown = widgets.Dropdown(options=instruments, description='Instrument:' )
slider = widgets.IntSlider(value=0, min=0, max=max(duration_sec-5,0), step=1, description='Start (s):')
seg_len = widgets.BoundedIntText(value=5, min=1, max=max(duration_sec,1), description='Length (s):')
play_button = widgets.Button(description='Play segment')

def play_segment(instrument, start_sec, segment_sec=5):
    audio_data = track.targets[instrument].audio
    start_sample = int(start_sec * sr)
    end_sample = int(min((start_sec + segment_sec) * sr, audio_data.shape[0]))
    segment = audio_data[start_sample:end_sample]
    print(f'Playing {instrument} from {start_sec}s to {start_sec+segment_sec}s')
    if segment.size == 0:
        print('Selected range is empty (check start time and length).')
        return
    if segment.ndim == 1:
        display(Audio(segment, rate=sr))
    elif segment.ndim == 2:
        display(Audio(segment.T, rate=sr))
    else:
        print(f'Unsupported audio shape for {instrument}: {segment.shape}')

def on_play_clicked(b):
    play_segment(dropdown.value, int(slider.value), int(seg_len.value))

play_button.on_click(on_play_clicked)
display(widgets.HBox([dropdown, seg_len, slider, play_button]))

In [51]:
# Import additional libraries for preprocessing
import librosa
import scipy.signal
from scipy.signal import stft, istft
import os
from pathlib import Path
import warnings

## Preprocessing Configuration

In [79]:
# Default preprocessing configuration
class PreprocessingConfig:
    def __init__(self):
        self.sample_rate = 44100
        self.chunk_len = 4.0  # seconds
        self.overlap = 0.0  # configurable to 50%
        self.n_fft = 4096  # fallback 2048
        self.hop_length = self.n_fft // 4
        self.window = 'hann'
        self.peak = 0.99
        self.eps = 1e-8
        self.band_edges_hz = [0, 1500, 6000, self.sample_rate // 2]
        self.pad_chunks = True
        
        # Augmentation flags
        self.enable_random_gain = False
        self.enable_remix = False
        self.enable_time_stretch = False
        self.enable_pitch_shift = False
        self.enable_add_noise = False
        
        # Target sources
        self.target_sources = ['vocals', 'drums', 'bass', 'other']

config = PreprocessingConfig()
print(f"Preprocessing config initialized:")
print(f"Sample rate: {config.sample_rate} Hz")
print(f"Chunk length: {config.chunk_len} seconds")
print(f"FFT size: {config.n_fft}")
print(f"Hop length: {config.hop_length}")
print(f"Band edges: {config.band_edges_hz} Hz")

Preprocessing config initialized:
Sample rate: 44100 Hz
Chunk length: 4.0 seconds
FFT size: 4096
Hop length: 1024
Band edges: [0, 1500, 6000, 22050] Hz


## Core Preprocessing Functions

In [53]:
# 1. Track loading & verification
def load_track(musdb_track):
    """
    Load track stems and verify all required sources are present.
    
    Args:
        musdb_track: musdb.Track object
        
    Returns:
        stems_dict: dict with stem audio data
        sr: sample rate
    """
    stems_dict = {}
    sr = musdb_track.rate
    
    # Extract stems
    for source_name in config.target_sources:
        if source_name in musdb_track.targets:
            stems_dict[source_name] = musdb_track.targets[source_name].audio
        else:
            print(f"Warning: Source '{source_name}' not found in track '{musdb_track.name}'")
            # Create silent stem if missing
            if len(stems_dict) > 0:
                # Use shape from existing stem
                ref_shape = next(iter(stems_dict.values())).shape
                stems_dict[source_name] = np.zeros(ref_shape, dtype=np.float32)
            else:
                print(f"Cannot create silent stem for '{source_name}' - no reference shape available")
                continue
    
    if not stems_dict:
        raise ValueError(f"No valid stems found in track '{musdb_track.name}'")
        
    print(f"Loaded track '{musdb_track.name}' with sources: {list(stems_dict.keys())}")
    return stems_dict, sr

# Test with first track
test_stems, test_sr = load_track(mus_train[0])
print(f"Sample rate: {test_sr} Hz")
for source, audio in test_stems.items():
    print(f"{source}: shape {audio.shape}, dtype {audio.dtype}")

Loaded track 'A Classic Education - NightOwl' with sources: ['vocals', 'drums', 'bass', 'other']
Sample rate: 44100 Hz
vocals: shape (300032, 2), dtype float64
drums: shape (300032, 2), dtype float64
bass: shape (300032, 2), dtype float64
other: shape (300032, 2), dtype float64


In [54]:
# 2. Resample + dtype conversion
def resample_audio(wav, orig_sr, target_sr):
    """
    Resample audio to target sample rate and convert to float32.
    
    Args:
        wav: audio array
        orig_sr: original sample rate
        target_sr: target sample rate
        
    Returns:
        wav_resampled: resampled audio as float32
    """
    if orig_sr == target_sr:
        return wav.astype(np.float32)
    
    # Resample using librosa
    if wav.ndim == 1:
        wav_resampled = librosa.resample(wav, orig_sr=orig_sr, target_sr=target_sr)
    else:
        # Handle stereo/multi-channel
        wav_resampled = np.array([
            librosa.resample(wav[:, ch], orig_sr=orig_sr, target_sr=target_sr) 
            for ch in range(wav.shape[1])
        ]).T
    
    # Convert to float32 and ensure range [-1.0, 1.0]
    wav_resampled = wav_resampled.astype(np.float32)
    
    # Clip to valid range if needed
    if np.max(np.abs(wav_resampled)) > 1.0:
        print(f"Warning: Audio clipped during resampling. Max value: {np.max(np.abs(wav_resampled))}")
        wav_resampled = np.clip(wav_resampled, -1.0, 1.0)
    
    return wav_resampled

# Test resampling
test_audio = test_stems['vocals'][:44100]  # 1 second
resampled = resample_audio(test_audio, test_sr, config.sample_rate)
print(f"Original shape: {test_audio.shape}, dtype: {test_audio.dtype}")
print(f"Resampled shape: {resampled.shape}, dtype: {resampled.dtype}")
print(f"Range: [{np.min(resampled):.3f}, {np.max(resampled):.3f}]")

Original shape: (44100, 2), dtype: float64
Resampled shape: (44100, 2), dtype: float32
Range: [-0.189, 0.223]


In [55]:
# 3. Normalization functions
def normalize_peak(wav, peak=0.99):
    """
    Peak normalize audio to specified peak value.
    
    Args:
        wav: audio array
        peak: target peak value (default 0.99)
        
    Returns:
        wav_norm: peak normalized audio
    """
    max_val = np.max(np.abs(wav))
    if max_val > 0:
        scale_factor = peak / max_val
        wav_norm = wav * scale_factor
    else:
        wav_norm = wav.copy()
    
    return wav_norm.astype(np.float32)

def rms_normalize(wav, target_db=-20):
    """
    RMS normalize audio to target dB level.
    
    Args:
        wav: audio array
        target_db: target RMS level in dB
        
    Returns:
        wav_norm: RMS normalized audio
    """
    # Calculate current RMS
    rms = np.sqrt(np.mean(wav**2))
    if rms > 0:
        # Convert target dB to linear scale
        target_rms = 10**(target_db / 20)
        scale_factor = target_rms / rms
        wav_norm = wav * scale_factor
    else:
        wav_norm = wav.copy()
    
    return wav_norm.astype(np.float32)

# Test normalization
test_segment = test_stems['vocals'][:44100]
peak_norm = normalize_peak(test_segment, config.peak)
rms_norm = rms_normalize(test_segment, target_db=-20)

print(f"Original: max={np.max(np.abs(test_segment)):.3f}, RMS={np.sqrt(np.mean(test_segment**2)):.3f}")
print(f"Peak normalized: max={np.max(np.abs(peak_norm)):.3f}")
print(f"RMS normalized: RMS={np.sqrt(np.mean(rms_norm**2)):.3f}")

Original: max=0.223, RMS=0.033
Peak normalized: max=0.990
RMS normalized: RMS=0.100


In [56]:
# 4. Mixture creation and level check
def make_mixture(stems_dict, normalize=True, headroom=0.95):
    """
    Create mixture by summing stems with optional scaling to prevent clipping.
    
    Args:
        stems_dict: dict of stem audio arrays
        normalize: whether to apply peak normalization
        headroom: headroom to maintain when scaling
        
    Returns:
        mixture_wav: summed mixture
        metadata: dict with per-stem RMS/peak info
    """
    # Sum all stems
    mixture = np.zeros_like(next(iter(stems_dict.values())), dtype=np.float32)
    
    metadata = {}
    for source_name, audio in stems_dict.items():
        mixture += audio.astype(np.float32)
        
        # Record per-stem statistics
        metadata[f'{source_name}_rms'] = float(np.sqrt(np.mean(audio**2)))
        metadata[f'{source_name}_peak'] = float(np.max(np.abs(audio)))
    
    # Check for clipping and scale if needed
    mixture_peak = np.max(np.abs(mixture))
    metadata['mixture_peak_before_scaling'] = float(mixture_peak)
    
    if mixture_peak > headroom:
        scale_factor = headroom / mixture_peak
        mixture *= scale_factor
        print(f"Scaled mixture by {scale_factor:.3f} to prevent clipping (peak was {mixture_peak:.3f})")
        
        # Also scale individual stems to maintain consistency
        for source_name in stems_dict:
            stems_dict[source_name] = stems_dict[source_name] * scale_factor
    
    # Optional peak normalization
    if normalize:
        mixture = normalize_peak(mixture, config.peak)
    
    metadata['mixture_peak_final'] = float(np.max(np.abs(mixture)))
    metadata['mixture_rms_final'] = float(np.sqrt(np.mean(mixture**2)))
    
    return mixture, metadata

# Test mixture creation
test_mixture, mix_metadata = make_mixture(test_stems)
print(f"Mixture shape: {test_mixture.shape}")
print(f"Mixture peak: {mix_metadata['mixture_peak_final']:.3f}")
print(f"Mixture RMS: {mix_metadata['mixture_rms_final']:.3f}")
for key, value in mix_metadata.items():
    if 'peak' in key or 'rms' in key:
        print(f"{key}: {value:.3f}")

Mixture shape: (300032, 2)
Mixture peak: 0.990
Mixture RMS: 0.205
vocals_rms: 0.063
vocals_peak: 0.616
drums_rms: 0.021
drums_peak: 0.229
bass_rms: 0.134
bass_peak: 0.489
other_rms: 0.096
other_peak: 0.528
mixture_peak_before_scaling: 0.858
mixture_peak_final: 0.990
mixture_rms_final: 0.205


In [74]:
# 5. Chunking (fixed-length segments)
def chunk_waveform(wav, chunk_len_sec, sr, overlap=0.0, pad=True):
    """
    Split waveform into fixed-length chunks.
    
    Args:
        wav: audio array
        chunk_len_sec: chunk length in seconds
        sr: sample rate
        overlap: overlap fraction (0.0 to 0.9)
        pad: whether to pad the last chunk
        
    Returns:
        list of (chunk_array, start_time) tuples
    """
    chunk_len_samples = int(chunk_len_sec * sr)
    hop_samples = int(chunk_len_samples * (1 - overlap))
    
    chunks = []
    start_sample = 0
    
    while start_sample < wav.shape[0]:
        end_sample = start_sample + chunk_len_samples
        
        if end_sample <= wav.shape[0]:
            # Full chunk
            chunk = wav[start_sample:end_sample]
        elif pad:
            # Pad last chunk
            chunk = np.zeros((chunk_len_samples,) + wav.shape[1:], dtype=wav.dtype)
            remaining_samples = wav.shape[0] - start_sample
            chunk[:remaining_samples] = wav[start_sample:]
        else:
            # Skip incomplete chunk
            break
        
        start_time = start_sample / sr
        chunks.append((chunk, start_time))
        
        start_sample += hop_samples
        
        # Prevent infinite loop
        if hop_samples == 0:
            break
    
    return chunks

# Test chunking
test_audio_long = test_stems['vocals'][:int(10 * test_sr)]  # 10 seconds
chunks = chunk_waveform(test_audio_long, config.chunk_len, test_sr, overlap=0.0, pad=True)
print(f"Created {len(chunks)} chunks from {len(test_audio_long)/test_sr:.1f}s audio")
for i, (chunk, start_time) in enumerate(chunks[:3]):
    print(f"Chunk {i}: shape {chunk.shape}, start time {start_time:.1f}s")

Created 2 chunks from 6.8s audio
Chunk 0: shape (176400, 2), start time 0.0s
Chunk 1: shape (176400, 2), start time 4.0s


In [58]:
# 6. Spectrogram computation
def compute_stft(wav, n_fft, hop_length, window='hann'):
    """
    Compute Short-Time Fourier Transform.
    
    Args:
        wav: audio waveform
        n_fft: FFT size
        hop_length: hop length
        window: window type
        
    Returns:
        complex_spec: complex-valued spectrogram
    """
    # Use scipy STFT for consistency
    if wav.ndim == 1:
        _, _, complex_spec = stft(wav, nperseg=n_fft, noverlap=n_fft-hop_length, window=window)
    else:
        # Handle multi-channel
        complex_specs = []
        for ch in range(wav.shape[1]):
            _, _, spec_ch = stft(wav[:, ch], nperseg=n_fft, noverlap=n_fft-hop_length, window=window)
            complex_specs.append(spec_ch)
        complex_spec = np.stack(complex_specs, axis=-1)
    
    return complex_spec

def magnitude_phase_from_complex(complex_spec):
    """
    Extract magnitude and phase from complex spectrogram.
    
    Args:
        complex_spec: complex-valued spectrogram
        
    Returns:
        mag: magnitude spectrogram
        phase: phase spectrogram
    """
    mag = np.abs(complex_spec)
    phase = np.angle(complex_spec)
    return mag.astype(np.float32), phase.astype(np.float32)

# Test STFT computation
test_chunk = chunks[0][0]
if test_chunk.ndim > 1:
    test_chunk = test_chunk[:, 0]  # Use first channel for mono test

complex_spec = compute_stft(test_chunk, config.n_fft, config.hop_length, config.window)
mag, phase = magnitude_phase_from_complex(complex_spec)

print(f"Test chunk shape: {test_chunk.shape}")
print(f"Complex spec shape: {complex_spec.shape}")
print(f"Magnitude shape: {mag.shape}")
print(f"Phase shape: {phase.shape}")
print(f"Magnitude range: [{np.min(mag):.3f}, {np.max(mag):.3f}]")
print(f"Phase range: [{np.min(phase):.3f}, {np.max(phase):.3f}]")

Test chunk shape: (176400,)
Complex spec shape: (2049, 174)
Magnitude shape: (2049, 174)
Phase shape: (2049, 174)
Magnitude range: [0.000, 0.052]
Phase range: [-3.142, 3.142]


In [59]:
# 7. Log compression (optional)
def log_compress_magnitude(mag, method='log1p', eps=1e-8):
    """
    Apply log compression to magnitude spectrogram.
    
    Args:
        mag: magnitude spectrogram
        method: 'log1p' or 'log10'
        eps: small constant for numerical stability
        
    Returns:
        log_mag: log-compressed magnitude
    """
    if method == 'log1p':
        log_mag = np.log1p(mag)
    elif method == 'log10':
        log_mag = 20 * np.log10(mag + eps)
    else:
        raise ValueError(f"Unknown compression method: {method}")
    
    return log_mag.astype(np.float32)

# Test log compression
log_mag_log1p = log_compress_magnitude(mag, method='log1p')
log_mag_log10 = log_compress_magnitude(mag, method='log10', eps=config.eps)

print(f"Original mag range: [{np.min(mag):.3f}, {np.max(mag):.3f}]")
print(f"Log1p compressed range: [{np.min(log_mag_log1p):.3f}, {np.max(log_mag_log1p):.3f}]")
print(f"Log10 compressed range: [{np.min(log_mag_log10):.1f}, {np.max(log_mag_log10):.1f}] dB")

Original mag range: [0.000, 0.052]
Log1p compressed range: [0.000, 0.050]
Log10 compressed range: [-159.2, -25.8] dB


In [60]:
# 8. Mask target creation
def compute_irm_masks(sources_mag_list, mixture_mag, eps=1e-8):
    """
    Compute Ideal Ratio Masks (IRM) for source separation.
    
    Args:
        sources_mag_list: list of source magnitude spectrograms
        mixture_mag: mixture magnitude spectrogram
        eps: small constant for numerical stability
        
    Returns:
        irm_masks: list of IRM masks clipped to [0,1]
    """
    irm_masks = []
    
    for source_mag in sources_mag_list:
        # IRM = source_mag / (mixture_mag + eps)
        irm = source_mag / (mixture_mag + eps)
        # Clip to [0, 1] range
        irm = np.clip(irm, 0.0, 1.0)
        irm_masks.append(irm.astype(np.float32))
    
    return irm_masks

def compute_cirm_masks(sources_complex, mixture_complex, eps=1e-8):
    """
    Compute Complex Ideal Ratio Masks (cIRM) for phase-aware models.
    
    Args:
        sources_complex: list of source complex spectrograms
        mixture_complex: mixture complex spectrogram
        eps: small constant for numerical stability
        
    Returns:
        cirm_masks: list of complex IRM masks
    """
    cirm_masks = []
    
    for source_complex in sources_complex:
        # cIRM = source_complex / (mixture_complex + eps)
        cirm = source_complex / (mixture_complex + eps)
        cirm_masks.append(cirm.astype(np.complex64))
    
    return cirm_masks

# Test mask computation - we need source spectrograms first
print("Computing spectrograms for all sources...")
test_mixture_chunk = chunks[0][0]
if test_mixture_chunk.ndim > 1:
    test_mixture_chunk = test_mixture_chunk[:, 0]

# Get a test chunk from all sources
test_sources_chunks = {}
for source_name in config.target_sources:
    source_chunks = chunk_waveform(test_stems[source_name], config.chunk_len, test_sr)
    source_chunk = source_chunks[0][0]
    if source_chunk.ndim > 1:
        source_chunk = source_chunk[:, 0]
    test_sources_chunks[source_name] = source_chunk

# Compute spectrograms for all sources
sources_complex = []
sources_mag = []

for source_name in config.target_sources:
    source_complex = compute_stft(test_sources_chunks[source_name], config.n_fft, config.hop_length)
    source_mag, _ = magnitude_phase_from_complex(source_complex)
    sources_complex.append(source_complex)
    sources_mag.append(source_mag)

# Compute mixture spectrogram
mixture_complex = compute_stft(test_mixture_chunk, config.n_fft, config.hop_length)
mixture_mag, _ = magnitude_phase_from_complex(mixture_complex)

# Compute IRM masks
irm_masks = compute_irm_masks(sources_mag, mixture_mag, config.eps)

print(f"Computed {len(irm_masks)} IRM masks")
for i, (source_name, mask) in enumerate(zip(config.target_sources, irm_masks)):
    print(f"{source_name} IRM: shape {mask.shape}, range [{np.min(mask):.3f}, {np.max(mask):.3f}]")

Computing spectrograms for all sources...
Computed 4 IRM masks
vocals IRM: shape (2049, 174), range [0.083, 1.000]
drums IRM: shape (2049, 174), range [0.000, 1.000]
bass IRM: shape (2049, 174), range [0.000, 1.000]
other IRM: shape (2049, 174), range [0.000, 1.000]
Computed 4 IRM masks
vocals IRM: shape (2049, 174), range [0.083, 1.000]
drums IRM: shape (2049, 174), range [0.000, 1.000]
bass IRM: shape (2049, 174), range [0.000, 1.000]
other IRM: shape (2049, 174), range [0.000, 1.000]


In [61]:
# 9. Band-splitting utilities
def split_bands(mag, sr, n_fft, band_edges_hz):
    """
    Split frequency axis into bands.
    
    Args:
        mag: magnitude spectrogram
        sr: sample rate
        n_fft: FFT size
        band_edges_hz: list of band edge frequencies in Hz
        
    Returns:
        bands: list of band magnitude spectrograms
    """
    # Convert Hz to bin indices
    freq_bins = np.linspace(0, sr/2, n_fft//2 + 1)
    band_indices = []
    
    for freq_hz in band_edges_hz:
        bin_idx = np.argmin(np.abs(freq_bins - freq_hz))
        band_indices.append(bin_idx)
    
    bands = []
    for i in range(len(band_indices) - 1):
        start_bin = band_indices[i]
        end_bin = band_indices[i + 1]
        band_mag = mag[start_bin:end_bin, :]
        bands.append(band_mag)
    
    return bands

def normalize_bands(bands, method='mean_std'):
    """
    Normalize bands using mean/std or min/max.
    
    Args:
        bands: list of band spectrograms
        method: 'mean_std' or 'min_max'
        
    Returns:
        normalized_bands: list of normalized bands
        normalization_stats: dict with stats for denormalization
    """
    normalized_bands = []
    normalization_stats = {}
    
    for i, band in enumerate(bands):
        if method == 'mean_std':
            mean_val = np.mean(band)
            std_val = np.std(band)
            normalized_band = (band - mean_val) / (std_val + config.eps)
            normalization_stats[f'band_{i}'] = {'mean': mean_val, 'std': std_val}
        elif method == 'min_max':
            min_val = np.min(band)
            max_val = np.max(band)
            normalized_band = (band - min_val) / (max_val - min_val + config.eps)
            normalization_stats[f'band_{i}'] = {'min': min_val, 'max': max_val}
        else:
            raise ValueError(f"Unknown normalization method: {method}")
        
        normalized_bands.append(normalized_band.astype(np.float32))
    
    return normalized_bands, normalization_stats

# Test band splitting
bands = split_bands(mixture_mag, config.sample_rate, config.n_fft, config.band_edges_hz)
normalized_bands, band_stats = normalize_bands(bands, method='mean_std')

print(f"Split into {len(bands)} frequency bands:")
for i, (band, norm_band) in enumerate(zip(bands, normalized_bands)):
    freq_start = config.band_edges_hz[i]
    freq_end = config.band_edges_hz[i+1]
    print(f"Band {i} ({freq_start}-{freq_end} Hz): shape {band.shape}")
    print(f"  Original range: [{np.min(band):.3f}, {np.max(band):.3f}]")
    print(f"  Normalized range: [{np.min(norm_band):.3f}, {np.max(norm_band):.3f}]")

Split into 3 frequency bands:
Band 0 (0-1500 Hz): shape (139, 174)
  Original range: [0.000, 0.052]
  Normalized range: [-0.629, 10.720]
Band 1 (1500-6000 Hz): shape (418, 174)
  Original range: [0.000, 0.013]
  Normalized range: [-0.422, 28.547]
Band 2 (6000-22050 Hz): shape (1491, 174)
  Original range: [0.000, 0.001]
  Normalized range: [-0.400, 19.627]


## Data Augmentation Functions

In [62]:
# 10. Augmentation functions
def random_gain(wav, gain_range_db=(-6, 6), seed=None):
    """
    Apply random gain augmentation.
    
    Args:
        wav: audio waveform
        gain_range_db: tuple of (min_db, max_db)
        seed: random seed for reproducibility
        
    Returns:
        augmented_wav: gain-augmented audio
    """
    if seed is not None:
        np.random.seed(seed)
    
    gain_db = np.random.uniform(gain_range_db[0], gain_range_db[1])
    gain_linear = 10**(gain_db / 20)
    
    return wav * gain_linear

def add_noise(wav, snr_range_db=(10, 30), seed=None):
    """
    Add white noise at specified SNR range.
    
    Args:
        wav: audio waveform
        snr_range_db: tuple of (min_snr, max_snr) in dB
        seed: random seed for reproducibility
        
    Returns:
        noisy_wav: audio with added noise
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Calculate signal power
    signal_power = np.mean(wav**2)
    
    # Target SNR
    target_snr_db = np.random.uniform(snr_range_db[0], snr_range_db[1])
    target_snr_linear = 10**(target_snr_db / 10)
    
    # Calculate noise power
    noise_power = signal_power / target_snr_linear
    
    # Generate and add noise
    noise = np.random.normal(0, np.sqrt(noise_power), wav.shape)
    
    return wav + noise.astype(wav.dtype)

def time_stretch(wav, sr, stretch_range=(0.9, 1.1), seed=None):
    """
    Apply time stretching without changing pitch.
    
    Args:
        wav: audio waveform
        sr: sample rate
        stretch_range: tuple of (min_rate, max_rate)
        seed: random seed for reproducibility
        
    Returns:
        stretched_wav: time-stretched audio
    """
    if seed is not None:
        np.random.seed(seed)
    
    stretch_rate = np.random.uniform(stretch_range[0], stretch_range[1])
    
    # Use librosa for time stretching
    stretched_wav = librosa.effects.time_stretch(wav, rate=stretch_rate)
    
    return stretched_wav

def pitch_shift(wav, sr, semitone_range=(-2, 2), seed=None):
    """
    Apply pitch shifting.
    
    Args:
        wav: audio waveform
        sr: sample rate
        semitone_range: tuple of (min_semitones, max_semitones)
        seed: random seed for reproducibility
        
    Returns:
        shifted_wav: pitch-shifted audio
    """
    if seed is not None:
        np.random.seed(seed)
    
    n_steps = np.random.uniform(semitone_range[0], semitone_range[1])
    
    # Use librosa for pitch shifting
    shifted_wav = librosa.effects.pitch_shift(wav, sr=sr, n_steps=n_steps)
    
    return shifted_wav

# Test augmentations
print("Testing augmentation functions...")
test_aug_audio = test_mixture_chunk[:44100]  # 1 second

# Random gain
gained_audio = random_gain(test_aug_audio, gain_range_db=(-3, 3), seed=42)
print(f"Original RMS: {np.sqrt(np.mean(test_aug_audio**2)):.3f}")
print(f"Gained RMS: {np.sqrt(np.mean(gained_audio**2)):.3f}")

# Add noise
noisy_audio = add_noise(test_aug_audio, snr_range_db=(15, 25), seed=42)
print(f"Noisy RMS: {np.sqrt(np.mean(noisy_audio**2)):.3f}")

print("Augmentation functions ready!")

Testing augmentation functions...
Original RMS: 0.046
Gained RMS: 0.042
Noisy RMS: 0.046
Augmentation functions ready!


## Data Saving and Validation

In [63]:
# 11. Save per-chunk artifacts
def save_chunk_npz(path, **fields):
    """
    Save chunk data to compressed .npz file.
    
    Args:
        path: output file path
        **fields: keyword arguments with data fields
    """
    # Ensure output directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Handle special cases for list fields
    processed_fields = {}
    for key, value in fields.items():
        if key == 'bands' and isinstance(value, list):
            # Save bands as separate arrays
            for i, band in enumerate(value):
                processed_fields[f'band_{i}'] = band
        elif key == 'sources_wav' and isinstance(value, list):
            # Save sources as separate arrays  
            for i, source in enumerate(value):
                processed_fields[f'source_{i}'] = source
        elif key == 'irm_masks' and isinstance(value, list):
            # Save IRM masks as separate arrays
            for i, mask in enumerate(value):
                processed_fields[f'irm_mask_{i}'] = mask
        else:
            processed_fields[key] = value
    
    # Save with compression
    np.savez_compressed(path, **processed_fields)
    print(f"Saved chunk to {path}")

def validate_chunk_fields(fields_dict):
    """
    Validate chunk fields for sanity checks.
    
    Args:
        fields_dict: dict with chunk data
        
    Returns:
        warnings: list of warning messages
    """
    warnings = []
    
    # Check for NaN/Inf values
    for field_name, data in fields_dict.items():
        if isinstance(data, np.ndarray):
            if np.any(np.isnan(data)):
                warnings.append(f"Found NaN values in {field_name}")
            if np.any(np.isinf(data)):
                warnings.append(f"Found Inf values in {field_name}")
        elif isinstance(data, list):
            # Check list elements
            for i, item in enumerate(data):
                if isinstance(item, np.ndarray):
                    if np.any(np.isnan(item)):
                        warnings.append(f"Found NaN values in {field_name}[{i}]")
                    if np.any(np.isinf(item)):
                        warnings.append(f"Found Inf values in {field_name}[{i}]")
    
    # Check IRM mask ranges
    if 'irm_masks' in fields_dict:
        for i, mask in enumerate(fields_dict['irm_masks']):
            if np.any(mask < 0) or np.any(mask > 1):
                warnings.append(f"IRM mask {i} contains values outside [0,1] range")
    
    # Check audio ranges
    for field_name in ['mixture_wav']:
        if field_name in fields_dict:
            data = fields_dict[field_name]
            if isinstance(data, np.ndarray):
                if np.max(np.abs(data)) > 1.1:
                    warnings.append(f"{field_name} contains values > 1.1")
    
    # Check sources_wav ranges
    if 'sources_wav' in fields_dict:
        for i, source in enumerate(fields_dict['sources_wav']):
            if isinstance(source, np.ndarray):
                if np.max(np.abs(source)) > 1.1:
                    warnings.append(f"sources_wav[{i}] contains values > 1.1")
    
    return warnings

# Test saving and validation
test_fields = {
    'mixture_wav': test_mixture_chunk,
    'sources_wav': [test_sources_chunks[name] for name in config.target_sources],
    'mag': mixture_mag,
    'phase': phase,
    'log_mag': log_mag_log1p,
    'irm_masks': irm_masks,
    'bands': bands,
    'metadata': {
        'sr': config.sample_rate,
        'start_time': 0.0,
        'track_name': 'test_track',
        'channel_count': 1,
        'n_fft': config.n_fft,
        'hop_length': config.hop_length
    }
}

# Validate fields
validation_warnings = validate_chunk_fields(test_fields)
if validation_warnings:
    print("Validation warnings:")
    for warning in validation_warnings:
        print(f"  - {warning}")
else:
    print("All fields passed validation!")

# Save test chunk
output_dir = Path("processed_chunks")
test_chunk_path = output_dir / "test_chunk_0.npz"
save_chunk_npz(test_chunk_path, **test_fields)

All fields passed validation!
Saved chunk to processed_chunks\test_chunk_0.npz
Saved chunk to processed_chunks\test_chunk_0.npz


In [64]:
# 12. Sanity checks and reconstruction
def reconstruct_from_mask(mixture_complex, irm_mask):
    """
    Reconstruct source audio using IRM and mixture phase.
    
    Args:
        mixture_complex: complex mixture spectrogram
        irm_mask: ideal ratio mask
        
    Returns:
        reconstructed_audio: reconstructed time-domain audio
    """
    # Apply mask to mixture
    reconstructed_complex = mixture_complex * irm_mask
    
    # Inverse STFT
    _, reconstructed_audio = istft(reconstructed_complex, 
                                   nperseg=config.n_fft, 
                                   noverlap=config.n_fft-config.hop_length)
    
    return reconstructed_audio.astype(np.float32)

def compute_si_sdr(reference, estimate, eps=1e-8):
    """
    Compute Scale-Invariant Signal-to-Distortion Ratio (SI-SDR).
    
    Args:
        reference: reference signal
        estimate: estimated signal
        eps: small constant for numerical stability
        
    Returns:
        si_sdr: SI-SDR in dB
    """
    # Ensure same length
    min_len = min(len(reference), len(estimate))
    reference = reference[:min_len]
    estimate = estimate[:min_len]
    
    # Zero-mean
    reference = reference - np.mean(reference)
    estimate = estimate - np.mean(estimate)
    
    # Compute scale factor
    alpha = np.sum(reference * estimate) / (np.sum(reference**2) + eps)
    
    # Scaled reference
    scaled_ref = alpha * reference
    
    # Compute SI-SDR
    signal_power = np.sum(scaled_ref**2)
    noise_power = np.sum((estimate - scaled_ref)**2)
    
    si_sdr = 10 * np.log10(signal_power / (noise_power + eps) + eps)
    
    return si_sdr

# Test reconstruction for each source
print("Testing oracle reconstruction...")
reconstruction_results = {}

for i, source_name in enumerate(config.target_sources):
    # Reconstruct using IRM
    reconstructed = reconstruct_from_mask(mixture_complex, irm_masks[i])
    
    # Compare with original
    original = test_sources_chunks[source_name]
    
    # Compute metrics
    si_sdr = compute_si_sdr(original, reconstructed)
    
    # Energy comparison
    original_energy = np.sum(original**2)
    reconstructed_energy = np.sum(reconstructed**2)
    energy_ratio = reconstructed_energy / (original_energy + config.eps)
    
    reconstruction_results[source_name] = {
        'si_sdr_db': si_sdr,
        'energy_ratio': energy_ratio,
        'original_rms': float(np.sqrt(np.mean(original**2))),
        'reconstructed_rms': float(np.sqrt(np.mean(reconstructed**2)))
    }
    
    print(f"{source_name}:")
    print(f"  SI-SDR: {si_sdr:.2f} dB")
    print(f"  Energy ratio: {energy_ratio:.3f}")
    print(f"  Original RMS: {reconstruction_results[source_name]['original_rms']:.3f}")
    print(f"  Reconstructed RMS: {reconstruction_results[source_name]['reconstructed_rms']:.3f}")

print(f"\nOracle reconstruction test completed!")

Testing oracle reconstruction...
vocals:
  SI-SDR: 103.06 dB
  Energy ratio: 1.000
  Original RMS: 0.074
  Reconstructed RMS: 0.074
drums:
  SI-SDR: -37.45 dB
  Energy ratio: 0.046
  Original RMS: 0.022
  Reconstructed RMS: 0.005
bass:
  SI-SDR: -40.13 dB
  Energy ratio: 0.002
  Original RMS: 0.131
  Reconstructed RMS: 0.006
other:
  SI-SDR: -45.58 dB
  Energy ratio: 0.124
  Original RMS: 0.052
  Reconstructed RMS: 0.018

Oracle reconstruction test completed!


## Complete Preprocessing Pipeline

In [65]:
# Complete preprocessing pipeline
def preprocess_track(musdb_track, config, output_dir, apply_augmentations=False, augmentation_seed=None):
    """
    Complete preprocessing pipeline for a single MUSDB track.
    
    Args:
        musdb_track: musdb.Track object
        config: PreprocessingConfig object
        output_dir: output directory for processed chunks
        apply_augmentations: whether to apply data augmentations
        augmentation_seed: seed for reproducible augmentations
        
    Returns:
        processing_summary: dict with processing statistics
    """
    print(f"Processing track: {musdb_track.name}")
    
    # 1. Load track
    stems_dict, orig_sr = load_track(musdb_track)
    
    # 2. Resample if needed
    if orig_sr != config.sample_rate:
        print(f"Resampling from {orig_sr} Hz to {config.sample_rate} Hz")
        for source_name in stems_dict:
            stems_dict[source_name] = resample_audio(stems_dict[source_name], orig_sr, config.sample_rate)
    
    # 3. Normalize stems
    for source_name in stems_dict:
        stems_dict[source_name] = normalize_peak(stems_dict[source_name], config.peak)
    
    # 4. Create mixture
    mixture, mix_metadata = make_mixture(stems_dict)
    
    # 5. Apply augmentations if enabled
    if apply_augmentations:
        if config.enable_random_gain:
            for source_name in stems_dict:
                stems_dict[source_name] = random_gain(stems_dict[source_name], seed=augmentation_seed)
            mixture, _ = make_mixture(stems_dict)
        
        if config.enable_add_noise:
            mixture = add_noise(mixture, seed=augmentation_seed)
    
    # 6. Chunk the audio
    mixture_chunks = chunk_waveform(mixture, config.chunk_len, config.sample_rate, 
                                   config.overlap, config.pad_chunks)
    
    # Chunk all sources
    sources_chunks = {}
    for source_name in config.target_sources:
        source_chunks = chunk_waveform(stems_dict[source_name], config.chunk_len, 
                                      config.sample_rate, config.overlap, config.pad_chunks)
        sources_chunks[source_name] = source_chunks
    
    # 7. Process each chunk
    chunk_count = 0
    processing_summary = {
        'track_name': musdb_track.name,
        'total_chunks': len(mixture_chunks),
        'processed_chunks': 0,
        'failed_chunks': 0,
        'warnings': [],
        'metadata': mix_metadata
    }
    
    for chunk_idx, (mixture_chunk, start_time) in enumerate(mixture_chunks):
        try:
            # Ensure we have corresponding source chunks
            sources_chunk_dict = {}
            sources_wav_list = []
            
            for source_name in config.target_sources:
                if chunk_idx < len(sources_chunks[source_name]):
                    source_chunk, _ = sources_chunks[source_name][chunk_idx]
                    sources_chunk_dict[source_name] = source_chunk
                    sources_wav_list.append(source_chunk)
                else:
                    # Handle case where we have fewer source chunks
                    print(f"Warning: Missing source chunk for {source_name} at index {chunk_idx}")
                    continue
            
            if len(sources_wav_list) != len(config.target_sources):
                print(f"Skipping chunk {chunk_idx} due to missing sources")
                continue
            
            # Convert to mono if multi-channel for STFT processing
            if mixture_chunk.ndim > 1:
                mixture_mono = mixture_chunk[:, 0]
                sources_mono = [chunk[:, 0] for chunk in sources_wav_list]
            else:
                mixture_mono = mixture_chunk
                sources_mono = sources_wav_list
            
            # 8. Compute spectrograms
            mixture_complex = compute_stft(mixture_mono, config.n_fft, config.hop_length, config.window)
            mixture_mag, mixture_phase = magnitude_phase_from_complex(mixture_complex)
            
            sources_complex = []
            sources_mag = []
            for source_chunk in sources_mono:
                source_complex = compute_stft(source_chunk, config.n_fft, config.hop_length, config.window)
                source_mag, _ = magnitude_phase_from_complex(source_complex)
                sources_complex.append(source_complex)
                sources_mag.append(source_mag)
            
            # 9. Compute log magnitude
            log_mag = log_compress_magnitude(mixture_mag, method='log1p')
            
            # 10. Compute IRM masks
            irm_masks = compute_irm_masks(sources_mag, mixture_mag, config.eps)
            
            # 11. Split into bands
            bands = split_bands(mixture_mag, config.sample_rate, config.n_fft, config.band_edges_hz)
            
            # 12. Prepare chunk data
            chunk_data = {
                'mixture_wav': mixture_chunk,
                'sources_wav': sources_wav_list,
                'mag': mixture_mag,
                'phase': mixture_phase,
                'log_mag': log_mag,
                'irm_masks': irm_masks,
                'bands': bands,
                'sr': config.sample_rate,
                'start_time': start_time,
                'track_name': musdb_track.name,
                'channel_count': mixture_chunk.shape[1] if mixture_chunk.ndim > 1 else 1,
                'n_fft': config.n_fft,
                'hop_length': config.hop_length,
                'chunk_index': chunk_idx
            }
            
            # 13. Validate chunk
            warnings = validate_chunk_fields(chunk_data)
            if warnings:
                processing_summary['warnings'].extend([f"Chunk {chunk_idx}: {w}" for w in warnings])
            
            # 14. Save chunk
            chunk_filename = f"{musdb_track.name}_chunk_{chunk_idx:03d}.npz"
            chunk_path = Path(output_dir) / chunk_filename
            save_chunk_npz(chunk_path, **chunk_data)
            
            processing_summary['processed_chunks'] += 1
            
        except Exception as e:
            print(f"Error processing chunk {chunk_idx}: {str(e)}")
            processing_summary['failed_chunks'] += 1
            processing_summary['warnings'].append(f"Chunk {chunk_idx}: Processing failed - {str(e)}")
    
    print(f"Completed processing {musdb_track.name}: {processing_summary['processed_chunks']} chunks processed, {processing_summary['failed_chunks']} failed")
    
    return processing_summary

## Unit Tests and Quality Assurance

In [66]:
# 13. Unit tests
def run_unit_tests():
    """
    Run automated unit tests on the preprocessing pipeline.
    
    Returns:
        test_results: dict with test results
    """
    test_results = {
        'passed': 0,
        'failed': 0,
        'tests': []
    }
    
    def add_test_result(test_name, passed, message=""):
        test_results['tests'].append({
            'name': test_name,
            'passed': passed,
            'message': message
        })
        if passed:
            test_results['passed'] += 1
        else:
            test_results['failed'] += 1
        print(f"{'✓' if passed else '✗'} {test_name}{': ' + message if message else ''}")
    
    try:
        print("Running unit tests...")
        
        # Test 1: Track loading
        test_track = mus_train[0]
        stems, sr = load_track(test_track)
        add_test_result("Track loading", 
                       len(stems) == len(config.target_sources) and sr > 0,
                       f"Loaded {len(stems)} sources at {sr} Hz")
        
        # Test 2: Resampling
        test_audio = stems['vocals'][:44100]
        resampled = resample_audio(test_audio, sr, config.sample_rate)
        add_test_result("Audio resampling",
                       resampled.dtype == np.float32 and np.max(np.abs(resampled)) <= 1.0,
                       f"Output dtype: {resampled.dtype}, max: {np.max(np.abs(resampled)):.3f}")
        
        # Test 3: Normalization
        normalized = normalize_peak(test_audio, config.peak)
        add_test_result("Peak normalization",
                       np.max(np.abs(normalized)) <= config.peak + 1e-6,
                       f"Peak: {np.max(np.abs(normalized)):.3f}")
        
        # Test 4: Mixture creation
        mixture, metadata = make_mixture(stems)
        add_test_result("Mixture creation",
                       mixture.shape == stems['vocals'].shape and 'mixture_peak_final' in metadata,
                       f"Shape: {mixture.shape}")
        
        # Test 5: Chunking
        chunks = chunk_waveform(mixture, config.chunk_len, config.sample_rate)
        expected_chunk_samples = int(config.chunk_len * config.sample_rate)
        add_test_result("Audio chunking",
                       len(chunks) > 0 and chunks[0][0].shape[0] == expected_chunk_samples,
                       f"Created {len(chunks)} chunks")
        
        # Test 6: STFT computation
        test_chunk = chunks[0][0]
        if test_chunk.ndim > 1:
            test_chunk = test_chunk[:, 0]
        complex_spec = compute_stft(test_chunk, config.n_fft, config.hop_length)
        mag, phase = magnitude_phase_from_complex(complex_spec)
        add_test_result("STFT computation",
                       complex_spec.shape[0] == config.n_fft // 2 + 1,
                       f"Spec shape: {complex_spec.shape}")
        
        # Test 7: IRM computation
        # Create dummy source spectrograms
        dummy_sources = [mag * 0.5, mag * 0.3, mag * 0.2, mag * 0.1]
        irm_masks = compute_irm_masks(dummy_sources, mag)
        all_in_range = all(np.all((mask >= 0) & (mask <= 1)) for mask in irm_masks)
        add_test_result("IRM mask computation",
                       len(irm_masks) == len(dummy_sources) and all_in_range,
                       f"Created {len(irm_masks)} masks, all in [0,1]: {all_in_range}")
        
        # Test 8: Band splitting
        bands = split_bands(mag, config.sample_rate, config.n_fft, config.band_edges_hz)
        add_test_result("Band splitting",
                       len(bands) == len(config.band_edges_hz) - 1,
                       f"Created {len(bands)} bands")
        
        # Test 9: Validation function
        test_fields = {
            'mixture_wav': test_chunk,
            'irm_masks': irm_masks,
            'mag': mag
        }
        warnings = validate_chunk_fields(test_fields)
        add_test_result("Field validation",
                       isinstance(warnings, list),
                       f"Validation returned {len(warnings)} warnings")
        
        # Test 10: Oracle reconstruction
        reconstructed = reconstruct_from_mask(complex_spec, irm_masks[0])
        add_test_result("Oracle reconstruction",
                       reconstructed.shape[0] > 0 and np.isfinite(reconstructed).all(),
                       f"Reconstructed shape: {reconstructed.shape}")
        
    except Exception as e:
        add_test_result("Unit tests execution", False, f"Error: {str(e)}")
    
    print(f"\nUnit test summary: {test_results['passed']} passed, {test_results['failed']} failed")
    return test_results

# Run unit tests
unit_test_results = run_unit_tests()

Running unit tests...
Loaded track 'A Classic Education - NightOwl' with sources: ['vocals', 'drums', 'bass', 'other']
✓ Track loading: Loaded 4 sources at 44100 Hz
✓ Audio resampling: Output dtype: float32, max: 0.223
✓ Peak normalization: Peak: 0.990
✓ Mixture creation: Shape: (300032, 2)
✓ Audio chunking: Created 2 chunks
✓ STFT computation: Spec shape: (2049, 174)
✓ IRM mask computation: Created 4 masks, all in [0,1]: True
✓ Band splitting: Created 3 bands
✓ Field validation: Validation returned 0 warnings
✓ Oracle reconstruction: Reconstructed shape: (177152,)

Unit test summary: 10 passed, 0 failed
Loaded track 'A Classic Education - NightOwl' with sources: ['vocals', 'drums', 'bass', 'other']
✓ Track loading: Loaded 4 sources at 44100 Hz
✓ Audio resampling: Output dtype: float32, max: 0.223
✓ Peak normalization: Peak: 0.990
✓ Mixture creation: Shape: (300032, 2)
✓ Audio chunking: Created 2 chunks
✓ STFT computation: Spec shape: (2049, 174)
✓ IRM mask computation: Created 4 masks

## Batch Preprocessing: Complete MUSDB18 Dataset

Both train and test sets will undergo the same preprocessing pipeline to ensure consistency. The only difference is that they'll be saved to separate directories to maintain the train/test split.

In [68]:
# Enhanced batch preprocessing with progress tracking and error handling
from tqdm import tqdm
import json
import time
from datetime import datetime

def preprocess_dataset_subset(tracks, subset_name, config, base_output_dir, apply_augmentations=False):
    """
    Preprocess a subset (train or test) of the MUSDB18 dataset.
    
    Args:
        tracks: list of musdb.Track objects
        subset_name: 'train' or 'test'
        config: PreprocessingConfig object
        base_output_dir: base output directory
        apply_augmentations: whether to apply data augmentations
        
    Returns:
        subset_summary: dict with detailed statistics
    """
    print(f"\n{'='*60}")
    print(f"Processing {subset_name.upper()} set: {len(tracks)} tracks")
    print(f"{'='*60}")
    
    # Create subset-specific output directory
    subset_output_dir = Path(base_output_dir) / subset_name
    subset_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Track processing statistics
    subset_summary = {
        'subset_name': subset_name,
        'total_tracks': len(tracks),
        'processed_tracks': 0,
        'failed_tracks': 0,
        'total_chunks': 0,
        'processed_chunks': 0,
        'failed_chunks': 0,
        'processing_time_seconds': 0,
        'track_summaries': [],
        'failed_track_names': [],
        'warnings': []
    }
    
    start_time = time.time()
    
    # Process each track with progress bar
    for track in tqdm(tracks, desc=f"Processing {subset_name} tracks"):
        try:
            track_summary = preprocess_track(
                track, 
                config, 
                subset_output_dir,
                apply_augmentations=apply_augmentations,
                augmentation_seed=42  # Fixed seed for reproducibility
            )
            
            # Update statistics
            subset_summary['processed_tracks'] += 1
            subset_summary['total_chunks'] += track_summary['total_chunks']
            subset_summary['processed_chunks'] += track_summary['processed_chunks']
            subset_summary['failed_chunks'] += track_summary['failed_chunks']
            subset_summary['track_summaries'].append(track_summary)
            subset_summary['warnings'].extend(track_summary['warnings'])
            
        except Exception as e:
            print(f"\nFATAL ERROR processing track '{track.name}': {str(e)}")
            subset_summary['failed_tracks'] += 1
            subset_summary['failed_track_names'].append(track.name)
            subset_summary['warnings'].append(f"FATAL: {track.name} - {str(e)}")
    
    subset_summary['processing_time_seconds'] = time.time() - start_time
    
    # Print subset summary
    print(f"\n{subset_name.upper()} SET SUMMARY:")
    print(f"Tracks processed: {subset_summary['processed_tracks']}/{subset_summary['total_tracks']}")
    print(f"Chunks processed: {subset_summary['processed_chunks']}/{subset_summary['total_chunks']}")
    print(f"Failed tracks: {subset_summary['failed_tracks']}")
    print(f"Processing time: {subset_summary['processing_time_seconds']:.1f} seconds")
    
    if subset_summary['failed_track_names']:
        print(f"Failed tracks: {', '.join(subset_summary['failed_track_names'])}")
    
    return subset_summary

def save_processing_report(train_summary, test_summary, config, base_output_dir):
    """
    Save a comprehensive processing report.
    
    Args:
        train_summary: training set processing summary
        test_summary: test set processing summary
        config: preprocessing configuration
        base_output_dir: base output directory
    """
    report = {
        'processing_date': datetime.now().isoformat(),
        'configuration': {
            'sample_rate': config.sample_rate,
            'chunk_len': config.chunk_len,
            'overlap': config.overlap,
            'n_fft': config.n_fft,
            'hop_length': config.hop_length,
            'window': config.window,
            'band_edges_hz': config.band_edges_hz,
            'target_sources': config.target_sources
        },
        'train_summary': train_summary,
        'test_summary': test_summary,
        'overall_summary': {
            'total_tracks': train_summary['total_tracks'] + test_summary['total_tracks'],
            'processed_tracks': train_summary['processed_tracks'] + test_summary['processed_tracks'],
            'failed_tracks': train_summary['failed_tracks'] + test_summary['failed_tracks'],
            'total_chunks': train_summary['total_chunks'] + test_summary['total_chunks'],
            'processed_chunks': train_summary['processed_chunks'] + test_summary['processed_chunks'],
            'failed_chunks': train_summary['failed_chunks'] + test_summary['failed_chunks'],
            'total_processing_time': train_summary['processing_time_seconds'] + test_summary['processing_time_seconds']
        }
    }
    
    # Save report as JSON
    report_path = Path(base_output_dir) / 'preprocessing_report.json'
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)
    
    print(f"\nProcessing report saved to: {report_path}")
    return report

print("Batch preprocessing functions defined.")

Batch preprocessing functions defined.


In [80]:
# Configure preprocessing settings for the full dataset
print("Configuring preprocessing for full MUSDB18 dataset...")

# Base output directory for all processed data
base_output_dir = Path("musdb18_processed")
base_output_dir.mkdir(exist_ok=True)

# You can modify these settings before running the batch processing
print("\nCurrent configuration:")
print(f"Sample rate: {config.sample_rate} Hz")
print(f"Chunk length: {config.chunk_len} seconds")
print(f"Overlap: {config.overlap} (0.0 = no overlap)")
print(f"FFT size: {config.n_fft}")
print(f"Apply augmentations: {config.enable_random_gain or config.enable_add_noise}")

# Estimate processing requirements
total_tracks = len(mus_train) + len(mus_test)
avg_track_duration = 6.0  # Average track duration in minutes
avg_chunks_per_track = int((avg_track_duration * 60) / config.chunk_len)
estimated_total_chunks = total_tracks * avg_chunks_per_track

print(f"\nDataset overview:")
print(f"Train tracks: {len(mus_train)}")
print(f"Test tracks: {len(mus_test)}")
print(f"Total tracks: {total_tracks}")
print(f"Estimated chunks: ~{estimated_total_chunks}")
print(f"Estimated disk space per chunk: ~15-20 MB")
print(f"Estimated total disk space: ~{estimated_total_chunks * 17.5 / 1024:.1f} GB")

print(f"\nOutput directory structure:")
print(f"  {base_output_dir}/")
print(f"  ├── train/          (training set chunks)")
print(f"  ├── test/           (test set chunks)")
print(f"  └── preprocessing_report.json")

print("\nReady to start batch processing!")
print("Run the next cell to begin processing the entire dataset.")

Configuring preprocessing for full MUSDB18 dataset...

Current configuration:
Sample rate: 44100 Hz
Chunk length: 4.0 seconds
Overlap: 0.0 (0.0 = no overlap)
FFT size: 4096
Apply augmentations: False

Dataset overview:
Train tracks: 94
Test tracks: 50
Total tracks: 144
Estimated chunks: ~12960
Estimated disk space per chunk: ~15-20 MB
Estimated total disk space: ~221.5 GB

Output directory structure:
  musdb18_processed/
  ├── train/          (training set chunks)
  ├── test/           (test set chunks)
  └── preprocessing_report.json

Ready to start batch processing!
Run the next cell to begin processing the entire dataset.


In [70]:
# Execute full dataset preprocessing
print(" STARTING FULL MUSDB18 DATASET PREPROCESSING")
print("=" * 80)

# Record overall start time
overall_start_time = time.time()

# Process training set
print("PHASE 1: Processing Training Set")
train_summary = preprocess_dataset_subset(
    tracks=mus_train,
    subset_name='train',
    config=config,
    base_output_dir=base_output_dir,
    apply_augmentations=False  # Set to True if you want augmentations for training
)

print("\n" + "="*80)

# Process test set  
print("PHASE 2: Processing Test Set")
test_summary = preprocess_dataset_subset(
    tracks=mus_test,
    subset_name='test',
    config=config,
    base_output_dir=base_output_dir,
    apply_augmentations=False  # Typically no augmentations for test set
)

# Calculate total processing time
total_time = time.time() - overall_start_time

print("\n" + "="*80)
print(" FULL DATASET PREPROCESSING COMPLETED!")
print("="*80)

# Print comprehensive summary
overall_summary = {
    'total_tracks': train_summary['total_tracks'] + test_summary['total_tracks'],
    'processed_tracks': train_summary['processed_tracks'] + test_summary['processed_tracks'],
    'failed_tracks': train_summary['failed_tracks'] + test_summary['failed_tracks'],
    'total_chunks': train_summary['total_chunks'] + test_summary['total_chunks'],
    'processed_chunks': train_summary['processed_chunks'] + test_summary['processed_chunks'],
    'failed_chunks': train_summary['failed_chunks'] + test_summary['failed_chunks']
}

print(f"\nOVERALL SUMMARY:")
print(f" Tracks processed: {overall_summary['processed_tracks']}/{overall_summary['total_tracks']}")
print(f" Chunks processed: {overall_summary['processed_chunks']}/{overall_summary['total_chunks']}")
print(f" Failed tracks: {overall_summary['failed_tracks']}")
print(f" Failed chunks: {overall_summary['failed_chunks']}")
print(f"  Total processing time: {total_time/3600:.1f} hours ({total_time/60:.1f} minutes)")

# Calculate success rates
track_success_rate = (overall_summary['processed_tracks'] / overall_summary['total_tracks']) * 100
chunk_success_rate = (overall_summary['processed_chunks'] / overall_summary['total_chunks']) * 100

print(f" Track success rate: {track_success_rate:.1f}%")
print(f" Chunk success rate: {chunk_success_rate:.1f}%")

# Save comprehensive report
print(f"\n Generating processing report...")
report = save_processing_report(train_summary, test_summary, config, base_output_dir)

# List output files
train_files = list((base_output_dir / 'train').glob("*.npz"))
test_files = list((base_output_dir / 'test').glob("*.npz"))

print(f"\n OUTPUT FILES:")
print(f"Training chunks: {len(train_files)} files in {base_output_dir}/train/")
print(f"Test chunks: {len(test_files)} files in {base_output_dir}/test/")
print(f"Processing report: {base_output_dir}/preprocessing_report.json")

# Estimate disk usage
if train_files and test_files:
    sample_file_size = train_files[0].stat().st_size / (1024*1024)  # MB
    total_size_mb = (len(train_files) + len(test_files)) * sample_file_size
    print(f" Estimated disk usage: {total_size_mb/1024:.1f} GB")

print(f"\n Dataset preprocessing completed successfully!")
print(f"Both train and test sets underwent identical preprocessing pipelines.")
print(f"Data is ready for source separation model training and evaluation.")

 STARTING FULL MUSDB18 DATASET PREPROCESSING
PHASE 1: Processing Training Set

Processing TRAIN set: 94 tracks


Processing train tracks:   0%|          | 0/94 [00:00<?, ?it/s]

Processing track: A Classic Education - NightOwl
Loaded track 'A Classic Education - NightOwl' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.556 to prevent clipping (peak was 1.710)
Loaded track 'A Classic Education - NightOwl' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.556 to prevent clipping (peak was 1.710)
Saved chunk to musdb18_processed\train\A Classic Education - NightOwl_chunk_000.npz
Saved chunk to musdb18_processed\train\A Classic Education - NightOwl_chunk_000.npz


Processing train tracks:   1%|          | 1/94 [00:02<03:45,  2.43s/it]

Saved chunk to musdb18_processed\train\A Classic Education - NightOwl_chunk_001.npz
Completed processing A Classic Education - NightOwl: 2 chunks processed, 0 failed
Processing track: ANiMAL - Clinic A
Loaded track 'ANiMAL - Clinic A' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.455 to prevent clipping (peak was 2.088)
Loaded track 'ANiMAL - Clinic A' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.455 to prevent clipping (peak was 2.088)
Saved chunk to musdb18_processed\train\ANiMAL - Clinic A_chunk_000.npz
Saved chunk to musdb18_processed\train\ANiMAL - Clinic A_chunk_000.npz


...

Saved chunk to musdb18_processed\test\Triviul feat. The Fiend - Widow_chunk_001.npz
Completed processing Triviul feat. The Fiend - Widow: 2 chunks processed, 0 failed
Processing track: We Fell From The Sky - Not You
Loaded track 'We Fell From The Sky - Not You' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.471 to prevent clipping (peak was 2.018)
Loaded track 'We Fell From The Sky - Not You' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.471 to prevent clipping (peak was 2.018)
Saved chunk to musdb18_processed\test\We Fell From The Sky - Not You_chunk_000.npz
Saved chunk to musdb18_processed\test\We Fell From The Sky - Not You_chunk_000.npz


Processing test tracks:  98%|█████████▊| 49/50 [02:26<00:02,  2.55s/it]

Saved chunk to musdb18_processed\test\We Fell From The Sky - Not You_chunk_001.npz
Completed processing We Fell From The Sky - Not You: 2 chunks processed, 0 failed
Processing track: Zeno - Signs
Loaded track 'Zeno - Signs' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.513 to prevent clipping (peak was 1.850)
Loaded track 'Zeno - Signs' with sources: ['vocals', 'drums', 'bass', 'other']
Scaled mixture by 0.513 to prevent clipping (peak was 1.850)
Saved chunk to musdb18_processed\test\Zeno - Signs_chunk_000.npz
Saved chunk to musdb18_processed\test\Zeno - Signs_chunk_000.npz


Processing test tracks: 100%|██████████| 50/50 [02:29<00:00,  2.98s/it]

Saved chunk to musdb18_processed\test\Zeno - Signs_chunk_001.npz
Completed processing Zeno - Signs: 2 chunks processed, 0 failed

TEST SET SUMMARY:
Tracks processed: 50/50
Chunks processed: 100/100
Failed tracks: 0
Processing time: 149.1 seconds

 FULL DATASET PREPROCESSING COMPLETED!

OVERALL SUMMARY:
 Tracks processed: 144/144
 Chunks processed: 288/288
 Failed tracks: 0
 Failed chunks: 0
  Total processing time: 0.1 hours (7.2 minutes)
 Track success rate: 100.0%
 Chunk success rate: 100.0%

 Generating processing report...

Processing report saved to: musdb18_processed\preprocessing_report.json

 OUTPUT FILES:
Training chunks: 188 files in musdb18_processed/train/
Test chunks: 100 files in musdb18_processed/test/
Processing report: musdb18_processed/preprocessing_report.json
 Estimated disk usage: 4.0 GB

 Dataset preprocessing completed successfully!
Both train and test sets underwent identical preprocessing pipelines.
Data is ready for source separation model training and evaluat

## Optional: Custom Preprocessing Configurations

You can modify the preprocessing configuration before running the batch processing to suit your specific needs:

In [78]:
# Optional: Modify preprocessing configuration
# Uncomment and modify any of these settings before running the full preprocessing

# Example configurations for different use cases:

# 1. High-resolution configuration (for better quality, more disk space)
# config.n_fft = 8192
# config.hop_length = config.n_fft // 8
# config.chunk_len = 8.0  # Longer chunks

# 2. Fast processing configuration (for quick experiments)
# config.n_fft = 2048
# config.hop_length = config.n_fft // 4
# config.chunk_len = 2.0  # Shorter chunks

# 3. Overlapping chunks configuration (for more training data)
# config.overlap = 0.5  # 50% overlap between chunks

# 4. Enable data augmentation for training set only
# config.enable_random_gain = True
# config.enable_add_noise = True

# 5. Different frequency bands configuration
# config.band_edges_hz = [0, 500, 2000, 8000, 22050]  # 4 bands instead of 3

print("Configuration options available above.")
print("Current settings:")
print(f"  FFT size: {config.n_fft}")
print(f"  Hop length: {config.hop_length}")  
print(f"  Chunk length: {config.chunk_len}s")
print(f"  Overlap: {config.overlap}")
print(f"  Frequency bands: {config.band_edges_hz}")
print(f"  Augmentations enabled: {any([config.enable_random_gain, config.enable_add_noise, config.enable_time_stretch, config.enable_pitch_shift])}")

print("\nTo modify settings, uncomment and edit the desired configuration above, then re-run this cell.")

Configuration options available above.
Current settings:
  FFT size: 4096
  Hop length: 1024
  Chunk length: 4.0s
  Overlap: 0.0
  Frequency bands: [0, 1500, 6000, 22050]
  Augmentations enabled: False

To modify settings, uncomment and edit the desired configuration above, then re-run this cell.
